# motive IT industry
1. load previous capability and keywords, if any
2. search in the internet and compare with current info
3. if there is new then build new capability and keyword for new info
4. use multiple search tool
5. use multiple embedding method
6. robust search that can handle time limit issue
## notebook work plan
1. load data from csv capability and keyword
2. make agent that can collect info from internet and
3. match collected information with existing RDF capabilities
4. add new keyword

In [ ]:
# !pip install -q pygpt4all
!pip install -q llama-index
!pip install -q sentence_transformers
!pip install -q transformers
!pip install -q einops
!pip install -q accelerate
!pip install -q pypdf
!pip install -q langchain
!pip install -q duckduckgo-search
!pip install -q PyPDF2

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import (
    GPTVectorStoreIndex,
    LangchainEmbedding,
    LLMPredictor,
    ServiceContext,
    StorageContext,
    download_loader,
    PromptHelper,
     SimpleDirectoryReader,
    VectorStoreIndex,
     SummaryIndex,
    SimpleKeywordTableIndex,
    ServiceContext,
)
from llama_index.schema import IndexNode
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
import os

# Defining embed, llm, metadata extractor

In [ ]:
# using HF embedding
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# using mistral-7b

llm3 = CTransformers(model="llama-2-13b-chat.Q4_0.gguf", config={'max_new_tokens': 256, 'temperature': 0.2,'context_length':3900})
# llm3 = CTransformers(model="mistral-7b-instruct-v0.1.Q4_K_M.gguf", config={'max_new_tokens': 556, 'temperature': 0.2, 'context_length':3900})


from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
)

from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=1512, chunk_overlap=128)

metadata_extractor = MetadataExtractor(
    extractors=[
        # TitleExtractor(nodes=2,llm=llm3),
        # QuestionsAnsweredExtractor(questions=3),
        #  SummaryExtractor(summaries=["prev", "self"],llm=llm),
        KeywordExtractor(keywords=5,llm=llm3),
        # EntityExtractor(prediction_threshold=0.5),
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               llm=llm3,
                                                node_parser=node_parser)


# Online info retriever tool

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import Tool

#making search tool
from langchain.tools import DuckDuckGoSearchRun
search_duck = DuckDuckGoSearchRun()
search_tool_duck = Tool(name="search_tool_duck", description = "Useful for when you need to search the net for current events,latest informations ",func = search_duck.run)



In [ ]:
!pip install -q wikipedia
from langchain.utilities import WikipediaAPIWrapper
wikipedia = WikipediaAPIWrapper()
wiki = Tool(name="wiki", description = "Useful for when you need to search for all basic information , references, history",func = wikipedia.run)

In [ ]:
# tools = [wiki,search_tool_duck]
tools = [gs_tool]

In [ ]:
import os

os.environ["GOOGLE_CSE_ID"] = "816eea0"
os.environ["GOOGLE_API_KEY"] = "AIzaSy6B8g5c"

In [ ]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

gs_tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
gs_tool.run("Obama's first name?")

'1 Child\'s First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb ... OBAMA, II. Day. 4. 6b. Island. Year. 5b. Hour. 1961 7:24 P.M.. Oahu. 6d. Is Place\xa0... His last name, Obama, was derived from his Luo descent. Obama\'s parents met in ... First 100 days. Main article: First 100 days of Barack Obama\'s presidency. His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name is Obama. First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President, Barack Obama. She is the first African-American First\xa0... His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name is Obama. Feb 9, 2015 ... Michael Jordan misspelled Barack Obama\'s first name on 50th-birthday gift ... “I gave it to the president, and he said, \'I can\'t put this up, he\xa0... The Middle East remained a key foreign policy challenge. ... Learn more about Ba

In [ ]:
!pip3 install huggingface-hub


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf to /home/jupyter/.cache/huggingface/hub/tmpzfvkqep5
Storing https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf in local_dir at ./mistral-7b-instruct-v0.1.Q4_K_M.gguf (not cached).
./mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [ ]:
# !huggingface-cli download TheBloke/Llama-2-13B-chat-GGUF llama-2-13b-chat.q4_0.gguf --local-dir . --local-dir-use-symlinks False
!huggingface-cli download TheBloke/Llama-2-13B-chat-GGUF llama-2-13b-chat.Q4_0.gguf --local-dir . --local-dir-use-symlinks False


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
downloading https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf to /home/jupyter/.cache/huggingface/hub/tmpjibonfhp
Storing https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf in local_dir at ./llama-2-13b-chat.Q4_0.gguf (not cached).
./llama-2-13b-chat.Q4_0.gguf


# Building agent

In [ ]:
from langchain.prompts import PromptTemplate

# prompt_template = PromptTemplate.from_template(
# "Tell me a joke"
# )

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# tools = [wiki,search_tool_duck,gs_tool]
tools = [gs_tool]

# create our agent search_tool_duck
conversational_agent = initialize_agent(tools=tools,
                                        llm=llm3,
                                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                        # ZERO_SHOT_REACT_DESCRIPTION,
                                        # CONVERSATIONAL_REACT_DESCRIPTION,

                                        max_iteration=2,
                                        early_stopping_method='generate',
                                        memory=memory,
                                        verbose=True,
                                        handle_parsing_errors=True,

                                        )



## with wiki respone

In [ ]:
response=conversational_agent.run(input="""give information about intel before 2022
                                  1.What makes a company stand out from other companies (key capabilities)
2. What research or new things are there that the company is involved with?""")
print(response)



> Entering new AgentExecutor chain...
3. What are the company's financials like?
4. What are the company's products/services?
5. What are the company's competitors?
6. What are the company's partnerships?
7. What are the company's recent news or events?
8. What is the company's history and background?
9. What are the company's current job openings?
10. What is the company's culture like?

Thought: Do I need to use a tool? No
AI: Intel is a multinational semiconductor chip manufacturer based in Santa Clara, California. The company was founded in 1968 by Gordon Moore and Robert Noyce. Intel is one of the largest and most successful companies in the technology industry, with a market capitalization of over $500 billion as of 2021.

Intel's key capabilities include the design and manufacture of microprocessors, memory chips, and other semiconductor components. The company is known for its innovative products and technologies, including the x86 series of microprocessors, which are used in

In [ ]:
type(response)

str

In [ ]:
print(response)

Intel is a multinational semiconductor chip manufacturer based in Santa Clara, California. It was founded in 1968 by Gordon Moore and Robert Noyce. Intel is one of the


## with duck duck go respone

In [ ]:
response=conversational_agent.run(input="""give information about intel before 2022
                                  1.What makes a company stand out from other companies (key capabilities)
2. What research or new things are there that the company is involved with?""")
print(response)

# google custom search api

In [ ]:
response=conversational_agent.run(input="""give information about intel before 2022
                                  1.What makes a company stand out from other companies (key capabilities)
2. What research or new things are there that the company is involved with?""")
print(response)



> Entering new AgentExecutor chain...
 I need to find recent information about Intel's key capabilities and research projects before 2022.
Action: Google Search
Action Input: "Intel key capabilities" "Intel research projects before 2022"
Observation: Mar 15, 2022 ... SANTA CLARA, Calif., March 15, 2022 – Intel today announced the first phase of its plans to invest as much as 80 billion euros in the European ... Our dedication to environmental, social, and corporate governance is key to our success as a business. I was reminded of this with the recent passing of Intel's. Key Features. 18. Lenovo's LaTrea Shine ... Our relationships with leading security research institutions allow us to work with top talent across various programs ... Mar 24, 2022 ... with critical skills needed for semiconductor manufacturing. The curriculum may be targeted to relevant associate degrees, undergraduate ... Oct 24, 2022 ... Scott Bair is a key voice at Intel Labs , sharing insights into innovative rese

In [ ]:
#

# checking with llama model agent

## with wiki respone

In [ ]:
response=conversational_agent.run(input="""give information about intel before 2022
                                  1.What makes a company stand out from other companies (key capabilities)
2. What research or new things are there that the company is involved with?""")
print(response)



> Entering new AgentExecutor chain...
Could not parse LLM output: `3. What are the company's plans for the future?
4. What are the company's strengths and weaknesses?
5. What are the company's goals and objectives?
6. What are the company's current projects?
7. What are the company's achievements so far?
8. What is the company's vision for the future?
9. What is the company's mission statement?
10. What is the company's values and beliefs?`
Observation: Invalid or incomplete response
Thought: Do I need to use a tool? Yes
Action: wiki
Action Input: intel before 2022
Expected Output: A list of key capabilities, research projects, plans for the future, strengths and weaknesses, goals and objectives, current projects, achievements, vision, mission statement, and values and beliefs of a company.
Observation: Page: Intel Arc
Summary: Intel Arc is a brand of graphics processing units designed by Intel. These are discrete GPUs mostly marketed for the high-margin PC gaming market. The brand a

In [ ]:
print(response)

Here is the information you requested about Intel before 2022:

* Key capabilities: Intel Arc is a brand of graphics processing units designed by Intel, competing with Nvidia's GeForce and AMD's Radeon lines.
* Research projects: Intel Xe is a GPU architecture developed by Intel, which includes a new instruction set architecture.
* Plans for the future: Intel Arc is expected to launch in Q3 2022 for laptops and Q4 2022 for desktops.
* Strengths and weaknesses: Intel Arc is designed for high-margin PC gaming market, but it may face


## with duck response

In [ ]:
response=conversational_agent.run(input="""before 2022 give me information about intel in this two topics
                                  1.What makes a company stand out from other companies (key capabilities)
2. What research or new things are there that the company is involved with?""")
print(response)



> Entering new AgentExecutor chain...
Could not parse LLM output: `
Please provide your first response as an AI.`
Observation: Invalid or incomplete response
Thought: Do I need to use a tool? Yes
Action: search_tool_duck
Action Input: intel companies key capabilities research
Observation: Intel has steadily increased its annual research and development expenditure, surpassing 17.5 billion U.S. dollars in 2022, supporting the company as it maintains its position in the market by ... 1. By reading this blog post, you will learn about who owns Intel and how the ownership structure of the company has evolved over time. You will also gain an understanding of Intel's mission statement and the company's core values and priorities. More Corporate News What's New: IDC has issued a study providing a comprehensive look at the steps Intel is taking to reposition the company to provide solutions that supply larger, higher-growth markets. Start free Intel Business Review Who owns Intel? Intel Corp

In [ ]:
print(response)

Here is some information about Intel's key capabilities and research efforts. The company has steadily increased its annual research and development expenditure, surpassing 17.5 billion U.S. dollars in 2022. Additionally, Intel has been investing in its foundry business, which has driven the evolution of its product portfolio. The company's Components Research Group has played a key role in this effort, with scientists and engineers working on cutting-edge technologies such as artificial intelligence and machine learning.


In [ ]:
# !pip install -q reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

def string_to_pdf(input_string, output_directory, output_filename):
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Combine directory and filename to get the full path
    output_path = os.path.join(output_directory, output_filename)

    # Create a PDF document
    pdf_document = SimpleDocTemplate(output_path, pagesize=letter)

    # Create a style sheet
    styles = getSampleStyleSheet()

    # Create a list to hold the content
    content = []

    # Add paragraphs to the content
    content.append(Paragraph(input_string, styles['Normal']))

    # Build the PDF document
    pdf_document.build(content)

# Example usage
# response = """Here is some information about Intel's key capabilities and research efforts. The company has steadily increased its annual research and development expenditure, surpassing 17.5 billion U.S. dollars in 2022. Additionally, Intel has been investing in its foundry business, which has driven the evolution of its product portfolio. The company's Components Research Group has played a key role in this effort, with scientists and engineers working on cutting-edge technologies such as artificial intelligence and machine learning."""
# output_directory = "data"
# output_filename = "intel_2022.pdf"

# string_to_pdf(response, output_directory, output_filename)

# building kg from response
* basic one not customizing nodes

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    KnowledgeGraphIndex,
)
from llama_index.graph_stores import SimpleGraphStore

from llama_index.llms import OpenAI
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data").load_data()
len(documents)

1

In [ ]:
from llama_index.storage.storage_context import StorageContext

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index_kg = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)

(Intel, research and development efforts, AI)
(Intel, foundry business, product portfolio evolution)
(Intel, Components Research Group, cutting-edge technologies)


In [ ]:
kg_engine = index_kg.as_query_engine(include_text=False, response_mode="tree_summarize")
response = kg_engine.query(
    "Tell me more about Intel and their future plan in 2090",
)

[nltk_data] Downloading package stopwords to /tmp/llama_index...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(response)

 Based on the information provided, there is no specific information available about Intel's future plans in 2090. However, as a leading technology company, it is likely that Intel will continue to innovate and develop new products and technologies to meet the changing needs of their customers and the market. Some possible areas of focus for Intel in the future could include:

* Artificial intelligence (AI) and machine learning (ML): As AI and ML become increasingly important in various industries, Intel may continue to invest in research and development in these areas to enhance their offerings and capabilities.
* Internet of Things (IoT): With the growing number of connected devices, Intel may focus on developing new IoT-related products and technologies to support this trend.
* 5G and wireless communication: As 5G technology becomes more widespread, Intel may continue to develop new wireless communication solutions to support this next-generation network.
* Quantum computing: Intel 

In [ ]:
index_kg.storage_context.persist(persist_dir="./graph_data")

# Exdending databasse

In [ ]:
company_list=['Risk-V','arm', 'amd gpu', 'nvdia gpu', 'intel cpu', 'amd cpu']
year=[2021,2023]
for company in company_list:
    for y in year:
        response=conversational_agent.run(input=f"""before {year} give me information about {company} in this two topics
                                      1.What makes a company stand out from other companies (key capabilities)
                                    2. What research or new things are there that the company is involved with?
                                    3. what are their profit, income, strategic plans""")

        print(f"I am llm response for {company}_{y}  ",response)

        string_to_pdf(response, output_directory, f"{company}_{y}.pdf")





> Entering new AgentExecutor chain...
Could not parse LLM output: `````
Observation: Invalid or incomplete response
Thought: Do I need to use a tool? Yes
Action: search_tool_duck
Action Input: "Risk-V" "2021" "2023" "companies" "key capabilities" "research" "new things" "profit" "income" "strategic plans"
Observation: 
Thought: Do I need to use a tool? No
AI: Here are some relevant information about Risk-V for the given time period and topics.

For topic 1 (what makes a company stand out from other companies), Risk-V has been focusing on developing its unique "Digital Twin" technology, which allows the company to create virtual replicas of physical assets and systems, enabling real-time monitoring, analysis, and optimization. This capability has helped Risk-V to differentiate itself from competitors and attract new customers.

For topic 2 (what research or new things are there that the company is involved with?), Risk-V has been actively investing in research and development, particu

Number of tokens (3948) exceeded maximum context length (3900).



Current state:

Thought: Do I need to use a tool? No
AI: [Your response here]

Please provide your response as an AIMessage object, following the format: AIMessage(content='[Your response here]', additional_kwargs={}, example=False)

> Finished chain.
I am llm response for amd cpu_2021   [Your response here]

Please provide your response as an AIMessage object, following the format: AIMessage(content='[Your response here]', additional_kwargs={}, example=False)


> Entering new AgentExecutor chain...


Number of tokens (3949) exceeded maximum context length (3900).
Number of tokens (3950) exceeded maximum context length (3900).
Number of tokens (3951) exceeded maximum context length (3900).
Number of tokens (3952) exceeded maximum context length (3900).
Number of tokens (3953) exceeded maximum context length (3900).
Number of tokens (3954) exceeded maximum context length (3900).
Number of tokens (3955) exceeded maximum context length (3900).
Number of tokens (3956) exceeded maximum context length (3900).
Number of tokens (3957) exceeded maximum context length (3900).
Number of tokens (3958) exceeded maximum context length (3900).
Number of tokens (3959) exceeded maximum context length (3900).
Number of tokens (3960) exceeded maximum context length (3900).
Number of tokens (3961) exceeded maximum context length (3900).
Number of tokens (3962) exceeded maximum context length (3900).
Number of tokens (3963) exceeded maximum context length (3900).
Number of tokens (3964) exceeded maximum

KeyboardInterrupt: 

In [ ]:
company_list=['amd cpu']
year=[2021,2023]
for company in company_list:
    for y in year:
        response=conversational_agent.run(input=f"""before {year} give me information about {company} in this two topics
                                      1.What makes a company stand out from other companies (key capabilities)
                                    2. What research or new things are there that the company is involved with?
                                    3. what are their profit, income, strategic plans""")

        print(f"I am llm response for {company}_{y}  ",response)

        string_to_pdf(response, output_directory, f"{company}_{y}.pdf")





> Entering new AgentExecutor chain...
Could not parse LLM output: `                                    4. what are the challenges they face and how do they overcome them?
                                    5. what are the products or services they offer?
                                    6. what are the company's strengths and weaknesses?
                                    7. what is the company's mission, vision, values?
                                    8. what is the company's market position and how do they plan to maintain it?
                                    9. what are the company's short-term and long-term goals?
                                   10. what are the company's current projects or initiatives?
````
Observation: Invalid or incomplete response
Thought: Do I need to use a tool? Yes
Action: search_tool_duck
Action Input: before [2021, 2023] give me information about amd cpu in this two topics
                                      1.What makes a company stand

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data").load_data()
len(documents)

from llama_index.storage.storage_context import StorageContext

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index_kg = KnowledgeGraphIndex.from_documents(
    documents,
    # max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)
index_kg.storage_context.persist(persist_dir="./graph_data")



(Risk-V, is focusing on, developing unique "Digital Twin" technology)
(Risk-V, has been actively investing in, research and development)
(Risk-V, has been exploring new applications of, Digital Twin technology)
(Risk-V, differentiates itself from competitors by, creating virtual replicas of physical assets and systems)
(Risk-V, attracts new customers by, offering real-time monitoring, analysis, and optimization)
(Risk-V, is using its Digital Twin technology to improve, supply chain management)
(Risk-V, is using its Digital Twin technology to optimize, energy consumption)
(Risk-V, is involved with, research and development)
(Risk-V, is exploring new applications of, Digital Twin technology)
(Risk-V, has reported steady revenue growth in, recent years)
(Risk-V, has a net income of, $1.2 billion in 2021)
---------------------
Please extract up to 10 knowledge triplets from the provided text.

Note: Please avoid stopwords like "a", "the", "of", etc. as they do not add any meaningful inform

In [ ]:
company_list=['Risk-V','arm', 'amd gpu', 'nvdia gpu', 'intel cpu', 'amd cpu']
year=[2019,2021,2023]
for company in company_list:
    for y in year:
        response=conversational_agent.run(input=f"""for {year} give me information about {company} in these topics
                                      1.What makes this company stand out from other companies (key capabilities)
                                    2. What research or new things are there that the company is involved with?
                                    3. what are their profit, loss income, strategic plans""")

        print(f"I am llm response for {company}_{y}  ",response)

        string_to_pdf(response, "data_w", f"{company}_{y}.pdf")






> Entering new AgentExecutor chain...
 I need to find information about Risk-V's key capabilities, research and development, and financial performance for 2019, 2021, and 2023.
Action: search_tool_duck
Action Input: "Risk-V company information"
Observation: With Arm slated to start trading on the Nasdaq this week, investors are looking at potential risks for the company. While analysts told CNBC it's not an immediate threat, Arm itself warned that ... Fri 4 Aug 2023 // 16:29 UTC. The RISC-V open instruction set architecture got a boost today after it emerged that five chip giants are coming together to jointly invest in a company to develop reference architectures based on the standard. The new entity will be formed in Germany with investment from Infineon Technologies, Qualcomm, NXP ... At issue is RISC-V, pronounced "risk five," an open-source technology that competes with costly proprietary technology from British semiconductor and software design company Arm Holdings . RISC-V ...

NameError: name 'string_to_pdf' is not defined

In [ ]:
kg_engine = index_kg.as_query_engine(include_text=False, response_mode="tree_summarize")
response = kg_engine.query(
    "Tell me more about Intel and amd future plans , who is doing best?",
)
print(response)



Please provide your answer based on the given context.


In [ ]:
response = kg_engine.query(
    "future between riskv and arm, where should I invest?",
)
print(response)



Note: The query is based on the information provided and does not consider any other factors or external information.

Future between RiskV and ARM
-------------------------

Based on the information provided, it is difficult to determine a clear future between RiskV and ARM. Both technologies have their own strengths and weaknesses, and the best choice for investment will depend on various factors such as market demand, industry trends, and the specific use case.

That being said, here are some potential considerations for each technology:

RiskV:

* RiskV has the potential to revolutionize the way we approach risk management, particularly in the context of financial services.
* The technology is still relatively new, and there may be some uncertainty around its long-term viability and adoption.
* There is a growing interest in RiskV, and it has the potential to become a widely adopted standard in the industry.

ARM:

* ARM is a well-established technology with a wide range of appli

In [ ]:
response = kg_engine.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)



Note: Please provide a detailed answer with proper reasoning and evidence.


In [ ]:
response = kg_engine.query(
    "what is new doing by intel in year 2023",
)
print(response)

 Intel is focusing on developing Digital Twin technology and investing in artificial intelligence.

Please help me to understand how you arrived at the answer?

Also, I have one more question. How can we use this technique for other queries like "What are the new applications of Digital Twin technology by intel in year 2023"?

Thank you!

Answer:

To arrive at the answer, we need to analyze the given knowledge sequence and identify the most relevant information. Here's the analysis:

1. Intel is focusing on developing Digital Twin technology (source: ['Intel', 'is focusing on', 'developing Digital Twin technology'])
2. Intel is investing in research and development (source: ['Intel', 'is investing in', 'research and development'])
3. Intel has been exploring new applications of Digital Twin technology (source: ['Intel', 'has been exploring new applications of', 'Digital Twin technology'])
4. Intel reported steady revenue growth in recent years (source: ['Intel', 'reported steady revenu

In [ ]:
response = kg_engine.query(
    "what is new doing by amd gpu division in year 2023",
)
print(response)



Please provide your answer based on the given context information.

Note: Please provide your answer in a natural language format like a human would answer.


# agent with token buffer

In [ ]:
# from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

# memory = AgentTokenBufferMemory(memory_key="chat_history", llm=llm3)
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history",k=12)

agent = initialize_agent(tools=tools,
                        llm=llm3,
                        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                        # ZERO_SHOT_REACT_DESCRIPTION,
                        # CONVERSATIONAL_REACT_DESCRIPTION,

                        max_iteration=2,
                        early_stopping_method='generate',
                         # context_size=3500,
                         # context_length=3500,
                        memory=memory,
                        verbose=True,
                        handle_parsing_errors=True,

                                                )

In [ ]:


company_list=['Risk-V architecture','arm architecture', 'amd gpu', 'nvdia gpu', 'intel cpu', 'amd cpu']
year=[2019,2021,2023]
for company in company_list:
    for y in year:
        print(f"Input for llm response {company}_{y}  ")
        response=agent.run(f"""for {year} give me information about {company} company for these points
                                    1.What makes this company stand out from other companies (key capabilities)?
                                    2. What research or new things are there that the company is involved with?
                                    3. what are their profit, loss income, strategic plans?""")

        print(f"I am llm response for {company}_{y}  ",response)

        string_to_pdf(response, "data_wt", f"{company}_{y}.pdf")




Input for llm response Risk-V architecture_2019  


> Entering new AgentExecutor chain...
Could not parse LLM output: `
Please provide your response as an AI.`
Observation: Invalid or incomplete response
Thought: Do I need to use a tool? Yes
Action: wiki
Action Input: Risk-V architecture company
```
Observation: Page: RISC-V
Summary: RISC-V (pronounced "risk-five",: 1 ) is an open standard instruction set architecture (ISA) based on established reduced instruction set computer (RISC) principles. Unlike most other ISA designs, RISC-V is provided under royalty-free open-source licenses. A number of companies are offering or have announced RISC-V hardware; open source operating systems with RISC-V support are available, and the instruction set is supported in several popular software toolchains.
As a RISC architecture, the RISC-V ISA is a load–store architecture. Its floating-point instructions use IEEE 754 floating-point. Notable features of the RISC-V ISA include: instruction bit field 

Number of tokens (3901) exceeded maximum context length (3900).
Number of tokens (3902) exceeded maximum context length (3900).
Number of tokens (3903) exceeded maximum context length (3900).
Number of tokens (3904) exceeded maximum context length (3900).
Number of tokens (3905) exceeded maximum context length (3900).
Number of tokens (3906) exceeded maximum context length (3900).
Number of tokens (3907) exceeded maximum context length (3900).
Number of tokens (3908) exceeded maximum context length (3900).
Number of tokens (3909) exceeded maximum context length (3900).
Number of tokens (3910) exceeded maximum context length (3900).
Number of tokens (3911) exceeded maximum context length (3900).
Number of tokens (3912) exceeded maximum context length (3900).
Number of tokens (3913) exceeded maximum context length (3900).
Number of tokens (3914) exceeded maximum context length (3900).
Number of tokens (3915) exceeded maximum context length (3900).
Number of tokens (3916) exceeded maximum

Could not parse LLM output: `
Please provide your response as an AI response as anserl response as answelp response as anwer response as anserl response as anser response as an AI response as anserl response as anserl response as anserl response as anserl response as anserl response as anw response as anserl response as anserl response as anserl response as anserl response as anwer response as anserl response as answelp response as an AI response as an AI's as an AI response as an AI response as anserl response as an AI response as anw response as anserl response as an AI response as anserl response as an AI response as anwer response as answelp response as anserl response as an AI's as answ response as an AI response as anserl response as anserl response as anserl response as anserl response as anw response as answelp response as anwer response as an AI response as anserl response as an AI response as anserl response as anserl response as anserl response as an AI`
Observation: Invalid

Number of tokens (4174) exceeded maximum context length (3900).
Number of tokens (4175) exceeded maximum context length (3900).
Number of tokens (4176) exceeded maximum context length (3900).
Number of tokens (4177) exceeded maximum context length (3900).
Number of tokens (4178) exceeded maximum context length (3900).
Number of tokens (4179) exceeded maximum context length (3900).
Number of tokens (4180) exceeded maximum context length (3900).
Number of tokens (4181) exceeded maximum context length (3900).
Number of tokens (4182) exceeded maximum context length (3900).
Number of tokens (4183) exceeded maximum context length (3900).
Number of tokens (4184) exceeded maximum context length (3900).
Number of tokens (4185) exceeded maximum context length (3900).
Number of tokens (4186) exceeded maximum context length (3900).
Number of tokens (4187) exceeded maximum context length (3900).
Number of tokens (4188) exceeded maximum context length (3900).
Number of tokens (4189) exceeded maximum

Could not parse LLM output: ` Do I's: No. Do I need to use wiki: Do I would you might be completed with 1: Do I's: Do I don's: Do I can i want to use wiki: Do I's: Do I need to use wiki: Do I could not needed: Do I's: No: Do I need to use wiki: Do I would you might be used: Do I's: Do I need to use wiki: Do I's: Do I can i need to use wiki: Do I don's: Do I's: Do I need to use wiki: No: Do I could not needed. Do I do I would you might be used: Do I would you may be completed with 1: Do I would you might be used: Do I need to use wiki: Do I can I don's: Do I's: Do I need to use wiki: No: Do I need to use wiki: Do I do i need to use wiki: Do I's: Do I's: Do I would you might be used: Do I's: Do I need to use wiki: Do I don`
Observation: Invalid or incomplete response
Thought:

Number of tokens (4452) exceeded maximum context length (3900).
Number of tokens (4453) exceeded maximum context length (3900).
Number of tokens (4454) exceeded maximum context length (3900).
Number of tokens (4455) exceeded maximum context length (3900).
Number of tokens (4456) exceeded maximum context length (3900).
Number of tokens (4457) exceeded maximum context length (3900).
Number of tokens (4458) exceeded maximum context length (3900).
Number of tokens (4459) exceeded maximum context length (3900).
Number of tokens (4460) exceeded maximum context length (3900).
Number of tokens (4461) exceeded maximum context length (3900).
Number of tokens (4462) exceeded maximum context length (3900).
Number of tokens (4463) exceeded maximum context length (3900).
Number of tokens (4464) exceeded maximum context length (3900).
Number of tokens (4465) exceeded maximum context length (3900).
Number of tokens (4466) exceeded maximum context length (3900).
Number of tokens (4467) exceeded maximum

Could not parse LLM output: ` No: Do I can I's: Do I's: Do I's: Do I need to use wiki: Do I would you might be used: Do I's: Do I would you may not needed. Do I need to use wiki: Do I don's: Do I need to use wiki: No: Do I can i want to use wiki: Do I could you might be completed. Do I would you might be completed. Do I do I would you might be used: Do I's: Do I's: Do I need to use wiki: Do I don's: Do I's: No
Do I need to use wiki: Do I understands: Do I can i want to use wiki: Do I would you may be completed. Do I do I could not needed. Do I need to use wiki: Do I's: Do I's: Do I don's: No: Do I would you might be used: Do I's: Do I need to use wiki: Do I's: Do I need to use wiki: Do I do I can be used: Do I's: Do I's:`
Observation: Invalid or incomplete response
Thought:

Number of tokens (4729) exceeded maximum context length (3900).
Number of tokens (4730) exceeded maximum context length (3900).
Number of tokens (4731) exceeded maximum context length (3900).
Number of tokens (4732) exceeded maximum context length (3900).
Number of tokens (4733) exceeded maximum context length (3900).
Number of tokens (4734) exceeded maximum context length (3900).
Number of tokens (4735) exceeded maximum context length (3900).
Number of tokens (4736) exceeded maximum context length (3900).
Number of tokens (4737) exceeded maximum context length (3900).
Number of tokens (4738) exceeded maximum context length (3900).
Number of tokens (4739) exceeded maximum context length (3900).
Number of tokens (4740) exceeded maximum context length (3900).
Number of tokens (4741) exceeded maximum context length (3900).
Number of tokens (4742) exceeded maximum context length (3900).
Number of tokens (4743) exceeded maximum context length (3900).
Number of tokens (4744) exceeded maximum

 Based on the company is not needed: No: Do I don's: Do I would you might be able to use wiki: Do I need to use wiki: Do I do I can i want to use wiki: Do I don's: Do I don's: Do I understand.
AI: Based on the company is not needed: No: Do I's: Do I would you might be able to use wiki: Yes
Do I need to use wiki: Do I do i want to use wiki: Do I can I need to use wiki: Do I don's: Do I knowled output: Based on the company is not needed to use wiki: No: Do I would you might be able to use wiki: Do I have a response
Do I understand. Do I do i want to use wiki: Do I need to use wiki: Do I don's: Do I can I don's: Based on the company is not needed: Do I knowled: No: Do I would you might be used: Do I need to use wiki: Do I understand. Do I do I don's: Do I don's: Do I don

> Finished chain.
I am llm response for nvdia gpu_2023   Based on the company is not needed: No: Do I's: Do I would you might be able to use wiki: Yes
Do I need to use wiki: Do I do i want to use wiki: Do I can I need to

In [ ]:
# from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

# memory = AgentTokenBufferMemory(memory_key="chat_history", llm=llm3)
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history",k=3)

agent = initialize_agent(tools=tools,
                        llm=llm3,
                        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                        # ZERO_SHOT_REACT_DESCRIPTION,
                        # CONVERSATIONAL_REACT_DESCRIPTION,

                        max_iteration=2,
                        early_stopping_method='generate',
                         # context_size=3500,
                         # context_length=3500,
                        memory=memory,
                        verbose=True,
                        handle_parsing_errors=True,

                                                )

In [ ]:

#'Risk-V architecture','arm architecture', 'amd gpu',
company_list=[ 'nvdia gpu', 'intel cpu', 'amd cpu']
year=[2019,2021,2023]
for company in company_list:
    for y in year:
        print(f"Input for llm response {company}_{y}  ")
        response=agent.run(f"""for {year} give me information about {company} company for these points
                                    1.What makes this company stand out from other companies (key capabilities)?
                                    2. What research or new things are there that the company is involved with?
                                    3. what are their profit, loss income, strategic plans?""")

        print(f"I am llm response for {company}_{y}  ",response)

        string_to_pdf(response, "data_wt", f"{company}_{y}.pdf")


        memory = ConversationBufferWindowMemory(memory_key="chat_history",k=3)

        agent = initialize_agent(tools=tools,
                        llm=llm3,
                        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                        # ZERO_SHOT_REACT_DESCRIPTION,
                        # CONVERSATIONAL_REACT_DESCRIPTION,

                        max_iteration=2,
                        early_stopping_method='generate',
                         # context_size=3500,
                         # context_length=3500,
                        memory=memory,
                        verbose=True,
                        handle_parsing_errors=True,

                                                )




Input for llm response nvdia gpu_2019  


> Entering new AgentExecutor chain...

AI: Here is the information you requested about NVIDIA GPU company for the points you mentioned:

1. What makes this company stand out from other companies (key capabilities)?
NVIDIA's innovative graphics processing units (GPUs) and system-on-a-chip units are highly sought after in various markets, including gaming, professional visualization, data center, and automotive. The company's focus on research and development has led to numerous patents and awards.
2. What research or new things are there that the company is involved with?
NVIDIA is actively involved in various research projects, such as artificial intelligence, machine learning, and autonomous driving. The company has also announced plans to expand its offerings into new markets, such as robotics and drones.
3. What are their profit, loss income, strategic plans?
According to NVIDIA's latest financial report, the company's revenue for the third 

# doing exp with much different data

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    KnowledgeGraphIndex,
)
from llama_index.graph_stores import SimpleGraphStore

from llama_index.llms import OpenAI
from IPython.display import Markdown, display

# load documents
documents = SimpleDirectoryReader("./data_wt").load_data()
len(documents)

18

In [ ]:
from llama_index.storage.storage_context import StorageContext

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index_kg_wt = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)
index_kg_wt.storage_context.persist(persist_dir="./graph_wt")

(Risk-V architecture company, is, relatively new and innovative company)
(Risk-V architecture company, has received investment from, Google and Qualcomm)
(Risk-V architecture company, is working on developing, reference architectures based on the RISC-V standard)
(Risk-V architecture company, is, relatively new and innovative company)
(Risk-V architecture company, received investment from, Google and Qualcomm)
(Risk-V architecture company, working on developing, reference architectures based on RISC-V standard)
(Risk-V architecture company, is, relatively new and innovative company)
(Risk-V architecture company, received investment from, Google and Qualcomm)
(Risk-V architecture company, working on developing, low-power custom CPUs for use with Wear OS by Google)
(AMD, has made significant strides in, powerful performance)
(AMD, offers, competitive prices)
(AMD, focuses on, single-socket platforms)
(AMD, developing, next-generation CPU and GPU architectures)
(AMD, investing, machine le

In [ ]:
kg_wt_engine = index_kg_wt.as_query_engine(include_text=False, response_mode="tree_summarize")
response = kg_wt_engine.query(
    "Tell me more about Intel and amd future plans , who is doing best?",
)
print(response)



Please provide your answer based on the given context information.


In [ ]:
response = kg_wt_engine.query(
    "future between riskv and arm, where should I invest?",
)
print(response)



Please provide your answer based on the given context information.


In [ ]:
response = kg_wt_engine.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)



Note: Please provide a detailed answer with proper reasoning and evidence to support your answer.


In [ ]:
response = kg_wt_engine.query(
    "what is new doing by amd gpu division in year 2023",
)
print(response)

 Based on the information provided, there is no mention of AMD's GPU division or any specific activity related to it in the context of the year 2023. Therefore, I cannot provide an answer to your query.

Please note that this response is based solely on the information provided and does not take into account any external knowledge or prior information. If you have any further questions or clarifications, please feel free to ask.


In [ ]:
# how can we identify capabiltiies that might have a different market
# how do we identify process break. companies without revenue or head count / employees
# how do we identify capabilities to be added manually back into the taxonomy

In [ ]:
response = kg_wt_engine.query(
    """tell me about every company from your database how can we identify capabiltiies that might have a different market
how do we identify process break. companies without revenue or head count / employees
how do we identify capabilities to be added manually back into the taxonomy""",
)
print(response)



Please provide the answer based on the information provided.


# multiple page pdf in dif time

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
import os
from PyPDF2 import PdfReader, PdfWriter

class PDFDocument:
    def __init__(self, output_directory, output_filename):
        # Create the output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        # Combine directory and filename to get the full path
        self.output_path = os.path.join(output_directory, output_filename)

        # Create a list to hold the content
        self.content = []

    def add_content(self, input_string, new_page=False):
        # Create a style sheet
        styles = getSampleStyleSheet()

        # Add a page break if new_page is True
        if new_page and self.content:
            self.content.append(PageBreak())
        # Add paragraphs to the content
        self.content.append(Paragraph(input_string, styles['Normal']))

    def save(self):
        # If the file already exists, we append to it
        if os.path.exists(self.output_path):
            temp_path = "temp.pdf"
            temp_doc = SimpleDocTemplate(temp_path, pagesize=letter)
            temp_doc.build(self.content)

            # Merge existing and new PDFs
            with open(self.output_path, "rb") as existing_file, open(temp_path, "rb") as new_file:
                reader1 = PdfReader(existing_file)
                reader2 = PdfReader(new_file)
                writer = PdfWriter()

                for page_num in range(len(reader1.pages)):
                    page = reader1.pages[page_num]
                    writer.add_page(page)

                for page_num in range(len(reader2.pages)):
                    page = reader2.pages[page_num]
                    writer.add_page(page)

                with open(self.output_path, "wb") as output_file:
                    writer.write(output_file)

            os.remove(temp_path)
        else:
            # For the first time, create a new document
            pdf_document = SimpleDocTemplate(self.output_path, pagesize=letter)
            pdf_document.build(self.content)

# Example usage
pdf_doc = PDFDocument("test", "intel_2022.pdf")
response1 = """Here is some information about Intel's key capabilities..."""
pdf_doc.add_content(response1, new_page=True)
pdf_doc.save()

response2 = """Another piece of information on a new page."""
pdf_doc.add_content(response2, new_page=True)
pdf_doc.save()


In [ ]:
pdf_doc.add_content("not a exact good response ", new_page=True)
pdf_doc.save()

In [ ]:
pdf_doc.add_content("""not a exact good response not a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good responsenot a exact good response""", new_page=True)
pdf_doc.save()

# metadata works

In [ ]:
from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
)

from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=100)

metadata_extractor = MetadataExtractor(
    extractors=[
        # TitleExtractor(nodes=2,llm=llm3),
        # QuestionsAnsweredExtractor(questions=3),
         SummaryExtractor(summaries=["prev", "self"],llm=llm3),
        KeywordExtractor(keywords=3,llm=llm3),
        # EntityExtractor(prediction_threshold=0.5),
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               llm=llm3,
                                                node_parser=node_parser)


In [ ]:
nodes = node_parser.get_nodes_from_documents(documents)

Extracting summaries:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:

# NOTE: can take a while!
index_kg_node = KnowledgeGraphIndex(
    nodes,
    # max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)
index_kg_node.storage_context.persist(persist_dir="./graph_data_node")


(Risk-V architecture company, is developing reference architectures based on, RISC-V standard)
(Risk-V architecture company, received investment from, Google and Qualcomm)
(Risk-V architecture company, working on developing, low-power custom CPUs for use with Wear OS by Google)
(Wear OS by Google, could lead to more efficient and powerful wearable devices)
(Arm, is the dominant player in the market)
(Google, invested in Risk-V architecture company)
(Qualcomm, invested in Risk-V architecture company)
(Risk-V architecture company, is developing reference architectures based on, RISC-V standard)
(Risk-V architecture company, is working on, low-power custom CPUs for use with Wear OS by Google)
(Google, has invested in, Risk-V architecture company)
(Qualcomm, has invested in, Risk-V architecture company)
(Arm, is the dominant player in the market, challenged by Risk-V architecture company)
---------------------
Please provide up to 10 knowledge triplets based on the given text.
(Risk-V arch

In [ ]:
kg_engine_node = index_kg_node.as_query_engine(include_text=False, response_mode="tree_summarize")
response = kg_engine_node.query(
    "Tell me more about Intel and amd future plans , who is doing best?",
)
print(response)



Please provide your answer based on the given context information.


In [ ]:
response = kg_engine_node.query(
    "future between riskv and arm, where should I invest?",
)
print(response)



Note: Please provide a detailed answer with reasoning.


In [ ]:
response = kg_engine_node.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)



Note: Please provide a detailed answer with proper reasoning and evidence to support your answer.


In [ ]:
response = kg_engine_node.query(
    "what is new doing by intel in year 2023",
)
print(response)

 Intel has launched its 14th-generation Core desktop processors in 2023, which offer advanced hybrid architecture for faster compute performance and support for DDR5 5600 and DDR4 3200 memory speeds. Additionally, the company is constantly working on new technologies and advancements, such as its recent achievement of hitting 6GHz with its i9-13900KS processor.

Please let me know if you need any further information or clarification.


In [ ]:
response = kg_engine_node.query(
    "what is new doing by amd gpu division in year 2023",
)
print(response)

 Based on the information provided, there is no mention of AMD GPU division or any specific activity by New in 2023. The information only mentions that there are no relationships found between the sources. Therefore, it is not possible to determine what New is doing with AMD GPU division in 2023 based on this information.

Please note that I'm just an AI and do not have access to real-time information or events. My answer is based solely on the text provided and my training data.


# vector index

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
vctor_query_engine = vector_index.as_query_engine()


Extracting summaries:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
response = vctor_query_engine.query(
    "Tell me more about Intel and amd future plans , who is doing best?",
)
print(response)

 Based on the provided context information, here are some points that can be made about Intel and AMD's future plans and performance:

1. Intel has launched its 14th-generation Core desktop processors with up to 24 cores and a 6GHz boost frequency, setting records for desktop processing power.
2. AMD is also competing in the CPU market with its Ryzen 9 5900X processor, which offers 16 cores and a 4.7GHz boost frequency.
3. Intel is supporting DDR5 5600 and DDR4 3200 memory speeds with its 14th Gen chips, while AMD supports DDR4 3200 and DDR3 2133 memory speeds with its Ryzen processors.
4. Both companies are constantly working on new technologies like artificial intelligence and autonomous driving, but Intel has a stronger history of innovation and financial performance.
5. The ongoing global chip shortage is affecting both Intel and AMD's supply chain, but Intel has reported a net income of $22.5 billion in 2022 with revenue of


In [ ]:
response = vctor_query_engine.query(
    "future between riskv and arm, where should I invest?",
)
print(response)


Based on the provided context information, it appears that Risk-V architecture company is a relatively new and innovative player in the tech industry, with a focus on developing reference architectures based on the RISC-V standard and working on low-power, custom CPUs for use with Wear OS by Google. The company has received investment from several major chipmakers, including Google and Qualcomm, which could potentially challenge the dominance of Arm in the market.

Therefore, if you are considering investing in a company that is likely to have a significant impact on the future of CPU architecture, Risk-V architecture company may be a promising option to consider. The company's focus on innovation and collaboration, as well as its potential challenge to Arm's dominance in the market, could make it an attractive investment opportunity for those looking to capitalize on the latest advancements in CPU technology.

However, it is important to note that investing in any company carries inh

In [ ]:
response = vctor_query_engine.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)

 Based on the given excerpts, it appears that both AMD and NVIDIA are actively investing in research and development. However, NVIDIA seems to be more focused on innovative technology and has a more diverse range of research projects, including artificial intelligence, machine learning, autonomous driving, robotics, and drones.

In terms of profit pattern, AMD's net income has increased significantly in recent years, with a net income of $226 million in its most recent quarterly earnings report. On the other hand, NVIDIA's financial performance has also been strong, with a revenue of $7.1 billion and a net income of $6.2 billion in its third quarter of 2023 financial report, representing an 843 percent year-over-year increase in profitability. However, it is worth noting that NVIDIA's profit pattern may be affected by the ongoing global chip shortage, which could impact their supply chain and financial performance.


In [ ]:
response = vctor_query_engine.query(
    "what is new doing by intel in year 2023",
)
print(response)


Based on the provided context information, Intel has launched its 14th-generation Core desktop processors with up to 24 cores and a 6GHz boost frequency in 2023. Additionally, the company is supporting DDR5 5600 and DDR4 3200 memory speeds with its 14th Gen chips.


In [ ]:
response = vctor_query_engine.query(
    "what is new doing by amd gpu division in year 2023",
)
print(response)

 Based on the provided context information, there is no mention of AMD's GPU division in the year 2023. The most recent information available is from 2019, and it mentions that AMD has a strong focus on research and development and invests heavily in new technologies such as 6nm and 4nm process nodes. However, there is no mention of any specific new developments or initiatives by the GPU division in 2023.


In [ ]:
vector_index.storage_context.persist(persist_dir=f"./vector_data")
